In [1]:
!pip install hopsworks

--- 

## <span style="color:#ff5f27;"> 🖍 Aggregate News Sentiments Each Day </span>

In [2]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5279
Connected. Call `.close()` to terminate connection gracefully.


In [26]:
market_news_fg = fs.get_or_create_feature_group(
    name = 'market_news_fg',
    version = 1
)

In [27]:
query = market_news_fg.select_all()
articles_df = query.read()

2023-01-13 01:15:23,643 INFO: USE `chyilin0_featurestore`
2023-01-13 01:15:24,197 INFO: SELECT `fg0`.`ticker` `ticker`, `fg0`.`publish_date` `publish_date`, `fg0`.`title` `title`, `fg0`.`body_text` `body_text`, `fg0`.`url` `url`, `fg0`.`text_w_puncts` `text_w_puncts`, `fg0`.`text_tokenized` `text_tokenized`, `fg0`.`text_w_stopwords` `text_w_stopwords`, `fg0`.`text_lemmatized` `text_lemmatized`, `fg0`.`text_stemmed` `text_stemmed`, `fg0`.`text_processed` `text_processed`, `fg0`.`neg` `neg`, `fg0`.`neu` `neu`, `fg0`.`pos` `pos`, `fg0`.`compound` `compound`, `fg0`.`predicted_class` `predicted_class`
FROM `chyilin0_featurestore`.`market_news_fg_1` `fg0`


In [28]:
articles_df

,ticker,publish_date,title,body_text,url,text_w_puncts,text_tokenized,text_w_stopwords,text_lemmatized,text_stemmed,text_processed,neg,neu,pos,compound,predicted_class
0,Apple,1672325820000,S&P 500 jumps as dip-buyers piled into tech,By Yasin Ebrahim Investing.com --The S&P 500 ...,https://www.investing.com//news/stock-market-n...,the sp 500 jumped thursday as dipbuying invest...,"[""the"",""sp"",""500"",""jumped"",""thursday"",""as"",""di...","[""sp"",""500"",""jumped"",""thursday"",""dipbuying"",""i...","[""sp"",""500"",""jumped"",""thursday"",""dipbuying"",""i...","[""sp"",""500"",""jump"",""thursday"",""dipbuy"",""invest...",sp 500 jump thursday dipbuy invest pil beatend...,0.095,0.849,0.056,-0.7500,0
1,Apple,1672399800000,Crypto Flipsider News – Binance Adds Apple Pay...,Read in the Digest: The world’s leading cryp...,https://www.investing.com//news/cryptocurrency...,ofstake as a significant step towards scaling ...,"[""ofstake"",""as"",""a"",""significant"",""step"",""towa...","[""ofstake"",""significant"",""step"",""towards"",""sca...","[""ofstake"",""significant"",""step"",""towards"",""sca...","[""ofstak"",""sign"",""step"",""toward"",""scal"",""netwo...",ofstak sign step toward scal network improv se...,0.082,0.905,0.013,-0.9738,0
2,Apple,1671719760000,Tech Stocks Head for Worst December Since 2002...,(Bloomberg) -- Technology stocks are headed fo...,https://www.investing.com//news/stock-market-n...,technology stocks are headed for their worst ...,"[""technology"",""stocks"",""are"",""headed"",""for"",""t...","[""technology"",""stocks"",""headed"",""worst"",""decem...","[""technology"",""stock"",""headed"",""worst"",""decemb...","[""technolog"",""stock"",""head"",""worst"",""decemb"",""...",technolog stock head worst decemb sint burst d...,0.106,0.790,0.104,-0.1872,1
3,Apple,1672816080000,Dow futures rise 90 pts; Fed minutes in the sp...,By Peter Nurse Investing.com -- U.S. stocks ar...,https://www.investing.com//news/stock-market-n...,us stocks are seen opening higher wednesday a...,"[""us"",""stocks"",""are"",""seen"",""opening"",""higher""...","[""us"",""stocks"",""seen"",""opening"",""higher"",""wedn...","[""u"",""stock"",""seen"",""opening"",""higher"",""wednes...","[""u"",""stock"",""seen"",""op"",""high"",""wednesday"",""a...",u stock seen op high wednesday attempt recov r...,0.060,0.893,0.046,-0.1154,1
4,Apple,1672422420000,Wall St ends 2022 with biggest annual drop sin...,By Echo Wang (Reuters) - U.S. stocks closed ou...,https://www.investing.com//news/economy/future...,us stocks closed out 2022 lower on friday cap...,"[""us"",""stocks"",""closed"",""out"",""2022"",""lower"",""...","[""us"",""stocks"",""closed"",""2022"",""lower"",""friday...","[""u"",""stock"",""closed"",""2022"",""lower"",""friday"",...","[""u"",""stock"",""clos"",""2022"",""low"",""friday"",""cap...",u stock clos 2022 low friday cap year sharp lo...,0.101,0.849,0.049,-0.9622,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13819,Apple,1541478000000,Ripple’s XRP Stages Rally on News of Apple Pay...,Ripple’s XRP took a turn for the better on Mon...,https://www.investing.com//news/cryptocurrency...,ripple s xrp took a turn for the better on mon...,"[""ripple"",""s"",""xrp"",""took"",""a"",""turn"",""for"",""t...","[""ripple"",""xrp"",""took"",""turn"",""better"",""monday...","[""ripple"",""xrp"",""took"",""turn"",""better"",""monday...","[""rippl"",""xrp"",""took"",""turn"",""bet"",""monday"",""r...",rippl xrp took turn bet monday ris 050 suggest...,0.054,0.787,0.159,0.8442,2
13820,Apple,1607059560000,Needham Stick to Their Buy Rating for Yext Inc,Needham analyst Ryan MacDonald reiterated a Bu...,https://www.investing.com//news/needham-stick-...,021 for the fourth quarter of 2020 the current...,"[""021"",""for"",""the"",""fourth"",""quarter"",""of"",""20...","[""021"",""fourth"",""quarter"",""2020"",""current"",""co...","[""021"",""fourth"",""quarter"",""2020"",""current"",""co...","[""021"",""four"",""quart"",""2020"","

In [29]:
keep_columns = ['ticker', 'publish_date', 'neg', 'neu', 'pos', 'compound']
sentiment_df = articles_df[keep_columns]
sentiment_df.head(5)

,ticker,publish_date,neg,neu,pos,compound
0,Apple,1672325820000,0.095,0.849,0.056,-0.7500
1,Apple,1672399800000,0.082,0.905,0.013,-0.9738
2,Apple,1671719760000,0.106,0.790,0.104,-0.1872
3,Apple,1672816080000,0.060,0.893,0.046,-0.1154
4,Apple,1672422420000,0.101,0.849,0.049,-0.9622


In [30]:
import pandas as pd
from datetime import datetime, timedelta

In [31]:
def time_2_datetime(x):
    
    dt_obj = datetime.fromtimestamp(x / 1000)
    return dt_obj

In [32]:
sentiment_df.publish_date = sentiment_df.publish_date.apply(time_2_datetime)
sentiment_df

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,ticker,publish_date,neg,neu,pos,compound
0,Apple,2022-12-29 15:57:00,0.095,0.849,0.056,-0.7500
1,Apple,2022-12-30 12:30:00,0.082,0.905,0.013,-0.9738
2,Apple,2022-12-22 15:36:00,0.106,0.790,0.104,-0.1872
3,Apple,2023-01-04 08:08:00,0.060,0.893,0.046,-0.1154
4,Apple,2022-12-30 18:47:00,0.101,0.849,0.049,-0.9622
...,...,...,...,...,...,...
13819,Apple,2018-11-06 05:20:00,0.054,0.787,0.159,0.8442
13820,Apple,2020-12-04 06:26:00,0.011,0.914,0.075,0.8910
13821,Apple,2020-10-28 15:45:00,0.049,0.787,0.165,0.9201
13822,Apple,2020-09-22 12:00:00,0.087,0.874,0.039,-0.3818


In [33]:
sentiment_df.sort_values(by='publish_date')

,ticker,publish_date,neg,neu,pos,compound
12956,Apple,2014-01-28 09:00:00,0.051,0.879,0.071,0.9233
10752,Apple,2014-02-10 14:05:00,0.067,0.885,0.048,-0.4588
10139,Apple,2014-02-11 12:25:00,0.088,0.831,0.081,0.2220
3089,Apple,2014-02-21 04:25:00,0.000,0.949,0.051,0.7906
7821,Apple,2014-03-05 10:45:00,0.041,0.931,0.029,-0.5050
...,...,...,...,...,...,...
16,Apple,2023-01-09 12:21:00,0.059,0.883,0.058,0.3612
11879,Apple,2023-01-09 14:18:00,0.049,0.878,0.073,0.9412
5743,Apple,2023-01-09 14:48:00,0.014,0.938,0.047,0.5106
74,Apple,2023-01-09 15:18:00,0.049,0.878,0.073,0.9412


In [34]:
acc_senti_df = sentiment_df.groupby([sentiment_df['publish_date'].dt.date, 'ticker']).agg({'neg': 'mean', 'neu': 'mean', 'pos': 'mean', 'compound': 'mean'}).reset_index()

acc_senti_df

,publish_date,ticker,neg,neu,pos,compound
0,2014-01-28,Apple,0.051000,0.8790,0.071000,0.923300
1,2014-02-10,Apple,0.067000,0.8850,0.048000,-0.458800
2,2014-02-11,Apple,0.088000,0.8310,0.081000,0.222000
3,2014-02-21,Apple,0.000000,0.9490,0.051000,0.790600
4,2014-03-05,Apple,0.041000,0.9310,0.029000,-0.505000
...,...,...,...,...,...,...
2566,2023-01-05,Apple,0.087000,0.8556,0.057600,-0.067380
2567,2023-01-06,Apple,0.073200,0.8430,0.083600,0.161420
2568,2023-01-07,Apple,0.120000,0.8190,0.061000,-0.900100
2569,2023-01-08,Apple,0.146000,0.8150,0.040000,-0.924000


In [36]:
def predicted_label(x):
  if x<=-0.5:
    return 0
  elif x>=0.5:
    return 2
  else:
    return 1

In [37]:
acc_senti_df2 = acc_senti_df
acc_senti_df2['predicted_class'] = acc_senti_df2['compound'].map(predicted_label)
acc_senti_df2

,publish_date,ticker,neg,neu,pos,compound,predicted_class
0,2014-01-28,Apple,0.051000,0.8790,0.071000,0.923300,2
1,2014-02-10,Apple,0.067000,0.8850,0.048000,-0.458800,1
2,2014-02-11,Apple,0.088000,0.8310,0.081000,0.222000,1
3,2014-02-21,Apple,0.000000,0.9490,0.051000,0.790600,2
4,2014-03-05,Apple,0.041000,0.9310,0.029000,-0.505000,0
...,...,...,...,...,...,...,...
2566,2023-01-05,Apple,0.087000,0.8556,0.057600,-0.067380,1
2567,2023-01-06,Apple,0.073200,0.8430,0.083600,0.161420,1
2568,2023-01-07,Apple,0.120000,0.8190,0.061000,-0.900100,0
2569,2023-01-08,Apple,0.146000,0.8150,0.040000,-0.924000,0


In [38]:
def datetime_2_int(x):
    
    dt_obj = datetime.strptime(str(x), '%Y-%m-%d')
    dt_obj = dt_obj.timestamp() * 1000
    return int(dt_obj)

In [39]:
acc_senti_df2.publish_date = acc_senti_df2.publish_date.apply(datetime_2_int)
acc_senti_df2 = acc_senti_df2.rename(columns={'publish_date': 'date', 'ticker': 'name'})
acc_senti_df2['name'] = acc_senti_df2['name'].str.upper()
acc_senti_df2

,date,name,neg,neu,pos,compound,predicted_class
0,1390863600000,APPLE,0.051000,0.8790,0.071000,0.923300,2
1,1391986800000,APPLE,0.067000,0.8850,0.048000,-0.458800,1
2,1392073200000,APPLE,0.088000,0.8310,0.081000,0.222000,1
3,1392937200000,APPLE,0.000000,0.9490,0.051000,0.790600,2
4,1393974000000,APPLE,0.041000,0.9310,0.029000,-0.505000,0
...,...,...,...,...,...,...,...
2566,1672873200000,APPLE,0.087000,0.8556,0.057600,-0.067380,1
2567,1672959600000,APPLE,0.073200,0.8430,0.083600,0.161420,1
2568,1673046000000,APPLE,0.120000,0.8190,0.061000,-0.900100,0
2569,1673132400000,APPLE,0.146000,0.8150,0.040000,-0.924000,0


In [40]:
market_sentiments_fg = fs.get_or_create_feature_group(
        name = 'market_sentiments_fg',
        description = 'Sentiment for market news aggragated each day',
        version = 1,
        primary_key = ['date','name'],
        online_enabled = True,
        event_time = 'date'
    )    

market_sentiments_fg.insert(acc_senti_df2)

Uploading Dataframe: 0.00% |          | Rows 0/2571 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/5279/jobs/named/market_sentiments_fg_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x1063a7f10>, None)

--- 

## <span style="color:#ff5f27;"> 🖍 Create Feature View and Training Data </span>

In [46]:
market_sentiments_fg = fs.get_or_create_feature_group(
    name = 'market_sentiments_fg',
    version = 1
)

stocks_fg = fs.get_or_create_feature_group(
    name = 'stocks_fg',
    version = 1
)

In [47]:
query = market_sentiments_fg.select_all().join(stocks_fg.select_all())

In [48]:
query.read()

2023-01-13 01:43:08,419 INFO: USE `chyilin0_featurestore`
2023-01-13 01:43:08,942 INFO: WITH right_fg0 AS (SELECT *
FROM (SELECT `fg1`.`date` `date`, `fg1`.`name` `name`, `fg1`.`neg` `neg`, `fg1`.`neu` `neu`, `fg1`.`pos` `pos`, `fg1`.`compound` `compound`, `fg1`.`predicted_class` `predicted_class`, `fg1`.`date` `join_pk_date`, `fg1`.`name` `join_pk_name`, `fg1`.`date` `join_evt_date`, `fg0`.`open` `open`, `fg0`.`high` `high`, `fg0`.`low` `low`, `fg0`.`close` `close`, `fg0`.`adj_close` `adj_close`, `fg0`.`volume` `volume`, `fg0`.`price_move` `price_move`, RANK() OVER (PARTITION BY `fg0`.`date`, `fg0`.`name`, `fg1`.`date` ORDER BY `fg0`.`date` DESC) pit_rank_hopsworks
FROM `chyilin0_featurestore`.`market_sentiments_fg_1` `fg1`
INNER JOIN `chyilin0_featurestore`.`stocks_fg_1` `fg0` ON `fg1`.`date` = `fg0`.`date` AND `fg1`.`name` = `fg0`.`name` AND `fg1`.`date` >= `fg0`.`date`) NA
WHERE `pit_rank_hopsworks` = 1) (SELECT `right_fg0`.`date` `date`, `right_fg0`.`name` `name`, `right_fg0`.`neg

,date,name,neg,neu,pos,compound,predicted_class,open,high,low,close,adj_close,volume,price_move
0,1672873200000,APPLE,0.087000,0.855600,0.057600,-0.067380,1,127.129997,127.769997,124.760002,125.019997,125.019997,80962700,1.0
1,1672786800000,APPLE,0.075500,0.886100,0.038300,-0.384690,1,126.889999,128.660004,125.080002,126.360001,126.360001,89113600,0.0
2,1672700400000,APPLE,0.081629,0.870114,0.048171,-0.431311,1,130.279999,130.899994,124.169998,125.070000,125.070000,112117500,1.0
3,1672354800000,APPLE,0.073444,0.884444,0.042000,-0.491044,1,128.410004,129.949997,127.430000,129.929993,129.929993,76960600,0.0
4,1672268400000,APPLE,0.090857,0.849286,0.060143,-0.109700,1,127.989998,130.479996,127.730003,129.610001,129.610001,75703700,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2109,1393974000000,APPLE,0.041000,0.931000,0.029000,-0.505000,0,18.961430,19.098213,18.897499,19.012857,16.861912,200062800,0.0
2110,1392937200000,APPLE,0.000000,0.949000,0.051000,0.790600,2,19.028214,19.091785,18.735714,18.758928,16.636713,278784800,1.0
2111,1392073200000,APPLE,0.088000,0.831000,0.081000,0.222000,1,18.950357,19.205357,18.910713,19.141430,16.975939,282256800,0.0
2112,1391986800000,APPLE,0.067000,0.885000,0.048000,-0.458800,1,18.523571,18.999643,18.500000,18.892500,16.755165,345559200,1.0


In [22]:
feature_view = fs.create_feature_view(
    name = 'stock_prediction_fv',
    version = 1,
    transformation_functions = {},
    query = query
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/5279/fs/5199/fv/stock_prediction_fv/version/1


In [41]:
feature_view = fs.get_feature_view(
    name = 'stock_prediction_fv',
    version = 1
)

In [42]:
feature_view.create_training_data()

Training dataset job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/5279/jobs/named/stock_prediction_fv_1_2_create_fv_td_13012023002724/executions


(2, <hsfs.core.job.Job at 0x14039f1c0>)

In [44]:
train_data = feature_view.get_training_data(2)[0]
train_data

,date,name,neg,neu,pos,compound,predicted_class,open,high,low,close,adj_close,volume,price_move
0,1424905200000,APPLE,0.065000,0.882667,0.052333,0.070433,1,32.197498,32.717499,31.652500,32.605000,29.465042,365150000,0.0
1,1488236400000,APPLE,0.060000,0.859500,0.080000,0.040450,1,34.270000,34.360001,34.174999,34.247501,32.189495,93931600,1.0
2,1516230000000,APPLE,0.075786,0.859929,0.064357,-0.109086,1,44.842499,45.025002,44.562500,44.814999,42.614571,124773600,0.0
3,1553554800000,APPLE,0.079667,0.841000,0.079444,0.204711,1,47.915001,48.220001,46.145000,46.697498,45.264591,199202000,1.0
4,1579129200000,APPLE,0.036429,0.900429,0.063000,0.277800,1,78.397499,78.925003,78.022499,78.809998,77.208374,108829200,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2109,1572822000000,APPLE,0.046333,0.880333,0.073333,0.441200,1,64.332497,64.462502,63.845001,64.375000,62.877968,103272000,0.0
2110,1585519200000,APPLE,0.071667,0.861333,0.067000,0.011733,1,62.685001,63.880001,62.349998,63.702499,62.556023,167976400,0.0
2111,1588284000000,APPLE,0.098600,0.820400,0.080600,0.014080,1,71.562500,74.750000,71.462502,72.267502,70.966866,240616800,1.0
2112,1590098400000,APPLE,0.026000,0.917000,0.057000,0.700300,2,78.942497,79.807503,78.837502,79.722504,78.499641,81803200,0.0


---
## Import history stock price